<h2><b>Langchain : Output -> Structured </b></h2>

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
llm=ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [ ]:
#TypedDict
from typing import TypedDict

class Person(TypedDict):
    name:str
    age:int

new_person = Person(name="Akshay", age=23)
print(type(new_person))
print(new_person)

<class 'dict'>
{'name': 'Akshay', 'age': 23}
Akshay


In [ ]:
# with_structured_output() + TypedDict works with OPENAI only 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated

load_dotenv()

# model= ChatGoogleGenerativeAI(model="gemini-2.0-flash")
model = ChatOpenAI()

class Review(TypedDict):
    summary:Annotated[str,'The Brief Summary of the Review']
    sentiment:Annotated[str,'Return Sentiment of the Review as Negative , Positive , Neutral']

structured_model = model.with_structured_output(Review)

res = structured_model.invoke("The new gaming laptop is outstanding! It has a sleek design, powerful performance, and excellent cooling. Perfect for both work and play.")

print(res)

In [42]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel


class Person(BaseModel):
    name:str
    age:int

parser=PydanticOutputParser(pydantic_object=Person)

prompt=PromptTemplate(
    input_variables=["Person"],
    template="Extract the name and age from the following sentence:\n\n{text}\n\n{format_instructions}",
    partial_variables ={"format_instructions":parser.get_format_instructions()}
)

input_text="My Name is Akshay and I am 23 Years old."

formatted_prompt=prompt.format(text=input_text)
res = llm.predict(formatted_prompt)

person = parser.parse(res)
print(person)

name='Akshay' age=23


In [ ]:
from pydantic import BaseModel
from typing import Optional

class Student(BaseModel):
    name:str = "Akshay"
    age:Optional[int ]=None

new_stu={'age':25}

st = Student(**new_stu)

print(st)

name='AK' age=25


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import Annotated
from pydantic import BaseModel,Field
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

model= ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

class Review(BaseModel):
    summary:str = Field(default=None,description="Summary of the Review in one line")
    sentiment:str=Field(default=None,description="Return Sentiment of the Review as strictly Negative , Positive , Neutral")
    product:str=Field(default=None ,description="Product Name about which the review is")

structured_model = model.with_structured_output(Review)

res = structured_model.invoke(
"""
The Samsung Galaxy Tab S9 stands out with its vibrant AMOLED display and sleek design, making it great for media consumption. The included S Pen is responsive and ideal for drawing or note-taking, and the overall performance is smooth for most tasks. However, some apps still feel poorly optimized for the tablet format, limiting the productivity experience. While the build quality feels premium, the device is quite expensive compared to competitors with similar specs. Bad Battery life is decent but can drain quickly during heavy usage. Overall, the Tab S9 offers a good user experience but comes with trade-offs that may affect its value depending on the user’s needs.
"""
)

print(res)

summary='Good user experience but expensive with some app optimization issues and battery concerns.' sentiment='Neutral' product='Samsung Galaxy Tab S9'
